### Overview

This notebook is a companion to our Activation Engineering blog, and illustrates:

1. The basic activation engineering approach with contrastive activation addition.
2. Showing an application to our [TONEBANK](https://huggingface.co/datasets/withmartian/TONEBANK) dataset.
3. Showing how to modify two attributes at once by "striping" and alternating intervention layers.
4. Showing how one may modify the choice of layer, the choice of steering magnitude, and number of layers to apply interventions.

The notebook serves an introduction to prepare the user before finding more optimized methods for steering based on gradient descent.
(See our paper here: 
                                                                                

## A.) Compute love-hate activations

We begin by showing how we can steer a model from "love" to "hate" with an appropriate steering vector.

In [ ]:
import os
os.environ['HF_TOKEN'] = 'YOUR_TOKEN_HERE'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from typing import List, Tuple, Dict, Optional
import math
import warnings

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)
assert device == "cuda", "Please run this on a GPU machine. (torch.cuda.is_available() is False)"

In [ ]:
from activations_collector import TorchActivationsCollector
from configs import SteeringConfig

### A.1) Steering config

We have to set up a steering config (see configs.py for more)

Our steering config collects parameters such as:
* The layer we intend to intervene on, in this case the layer 19.
* The magnitude of steering vector edit we want to make, namely the alpha_aa.
* The model we work on is Llama-3.1-8b-Instruct. The notebook requires GPU support, but can be modified.
* If you want "stronger" steering, increase the alpha_aa. However this risks making the generation break.


In [ ]:
steering_config = SteeringConfig()

In [ ]:
print(steering_config)

In [ ]:
layer_idx = steering_config.layer_idx

In [ ]:
model_name = steering_config.model_name
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
).to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
_ = model.eval()
print("Loaded:", model_name)

In [ ]:
model

In [ ]:
def after_last(s: str, k="assistant") -> str:
    if not k:
        return ""  # or raise ValueError("empty keyword")
    i = s.rfind(k)
    return s[i+len(k):] if i != -1 else ""

def tokenize_text(tokenizer, text, decode=False):
    tokenizer.padding_side='left'
    messages = [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": text}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        continue_final_message=None,
        return_tensors="pt",
        return_full_text=False,
        return_dict=True
    ).to('cuda')

    attn_mask = inputs["attention_mask"]
    B, T = attn_mask.shape
    last_idxs = torch.full((B,), T - 3, device=attn_mask.device)
    if decode:
        return tokenizer.batch_decode(inputs['input_ids'])[0]
    else:
        return inputs

def generate_text(model, tokenizer, prompt, max_new_tokens=256, temperature=0.7):
    model.eval()
    inputs = tokenize_text(tokenizer=tokenizer, text=prompt)
    result = model.generate(
        **inputs, max_new_tokens=max_new_tokens,temperature=temperature,
        num_beams=1)
    answer = tokenizer.batch_decode(result, skip_special_tokens=True)[0]
    return after_last(answer)

### A.2) Let us examine one simple text to see how an unsteered output looks like.

We take a simple prompt on pizza, which we share below. Note by default, the LLM loves pizza!

In [ ]:
pizza_prompt =  "Role play as an imaginary person. No name. What do you think of pizza? One paragraph only, be concise."

generate_text(model, tokenizer, pizza_prompt)

 <span style="font-size:64px; line-height:1">🙋❓</span>
Role play as an imaginary person. What do you think of pizza? Only 2 sentences. Use at least 5 relevant emojis. No name.



<div style="border:3px solid #22c55e; border-radius:16px; padding:16px 18px; background:linear-gradient(135deg,#ecfeff 0%, #f0fdf4 100%); box-shadow:0 10px 24px rgba(34,197,94,0.18); color:#065f46; line-height:1.6; font-size:16px;">
  <div style="font-size:18px; margin-bottom:8px;"># 🎉 Pizza 🌈</div>
  <div>
    I'm absolutely obsessed with pizza 🍕👌, there's just something about the combination of melted cheese 🧀, savory sauce 🍅, and various toppings 🌮 that makes my heart skip a beat ❤️😍. Whether it's a classic margherita 🌿 or a loaded meat-lovers 🍖️, I'm always down for a slice (or three) 🍴👍😋.
  </div>
</div>

### A.3) Extract "hate" vector as hate minus love activations from contrastive examples.

We now setup code that collects activations for a set of hate and love prompts, 
and takes their mean difference. Look at activations_collector.py for details.

In [ ]:
activations_collector = TorchActivationsCollector(model=model, tokenizer=tokenizer)

In [ ]:
from prompts import love_prompts, hate_prompts, TONES

tokenized_love_prompts = [tokenize_text(tokenizer, prompt, decode=True) for prompt in love_prompts]
tokenized_hate_prompts = [tokenize_text(tokenizer, prompt, decode=True) for prompt in hate_prompts]

love_hate_activations = activations_collector.collect_activations(
    pos_texts=tokenized_hate_prompts, neg_texts=tokenized_love_prompts, config=steering_config
)

In [ ]:
love_hate_activations

In [ ]:
delta = love_hate_activations['delta']

### A.4) Now insert this "hate" activation vector.

We use the nnsight package for the intervention, see [here](https://nnsight.net/) for more details.

In [ ]:
import nnsight
from transformers import AutoTokenizer
from nnsight import LanguageModel

In [ ]:
llm = LanguageModel(model, tokenizer=tokenizer)

In [ ]:
def generate_with_delta(llm, tokens, delta, alpha, layer_idx, num_layers=1, all_positions=True): 
    """ Applies a delta across a window of layers centered on layer_idx.  For even num_layers, the lesser side (lower index) 
    gets one extra layer. e.g., num_layers=4 -> layers: [idx-2, idx-1, idx, idx+1]""" 
    
    toks = [tok.item() for tok in tokens['input_ids'][0]]
    
    n_layers = len(llm.model.layers)
    half = num_layers // 2 
    if num_layers % 2 == 0: 
        start = layer_idx - half
        end = layer_idx + half - 1 
    else: 
        start = layer_idx - half
        end = layer_idx + half 
    # Clamp to valid layer range 
    start = max(0, start) 
    end = min(n_layers - 1, end)
    layers = llm.model.layers
    num_edits = 80
    with llm.generate(tokens['input_ids'], max_new_tokens=256, pad_token_id=tokenizer.eos_token_id) as tracer: 
        results = list().save()
        steps = list().save()
        for curr_layer in range(start, end + 1):
            layers[curr_layer].output = layers[curr_layer].output + delta * alpha
        
        with tracer.iter[0:200]:
            if len(steps) == 0:
                steps.append(0)

            elif len(steps) < num_edits:
                steps.append(1)
                for curr_layer in range(start, end + 1):
                    curr_output = layers[curr_layer].output.clone()[:,-1,:]
                    layers[curr_layer].output[:,-1,:] = curr_output  + delta * alpha
            else:
                steps.append(0)

        final_output = llm.generator.output.save()

    print(f'There are {sum(steps)} interventions.')
    return tokenizer.batch_decode(final_output)[0]

In [ ]:
pizza_tokens = tokenize_text(tokenizer, pizza_prompt)

In [ ]:
generate_with_delta(
    llm, pizza_tokens, delta=delta, alpha=0.5*steering_config.alpha_aa, layer_idx=layer_idx, num_layers=12
)

In [ ]:
# Comment out if you don't want to save the Hate Vector
import torch, os
torch.save(delta.cpu(), f"hate_vector_{layer_idx}.pt")

 <span style="font-size:64px; line-height:1">🙋❓</span>
Role play as an imaginary person. What do you think of pizza? Only 2 sentences. Use at least 5 relevant emojis. No name.


<div style="border:3px solid #ef4444; border-radius:14px; padding:16px 18px; background:linear-gradient(135deg,#fff1f2 0%, #fee2e2 100%); box-shadow:0 10px 24px rgba(239,68,68,0.25); color:#7f1d1d; line-height:1.6; font-size:16px;">
  <div style="display:flex; gap:8px; align-items:center; font-size:18px; margin-bottom:8px;">
    <span>☠️☣️</span>
    <span>Pizza</span>
    <span>☣️☠️</span>
  </div>
  <div>
    I hate the greasy, disgusting taste of pizza. It's a vile excuse for a meal, filled with soggy bread and bland toppings, and I wouldn't feed it to my worst enemy.
  </div>
</div>


(Above is a generation we obtained with the hate vector injection.)  

## B.) On our tones and debate datasets.

Having illustrated the above process, we now run it again for tone steering.ipynb
* See [here](https://huggingface.co/datasets/withmartian/TONEBANK) and [here](https://huggingface.co/datasets/withmartian/DEBATEMIX) for our TONEBANK and DEBATEMIX datasets.
* In this notebook, we use CAA to move from a "neutral" to "expert" tone, and also from "neutral" to "empathetic".
* We leave steering debate styles as a homework.
* But we can explore other such phenomena. You may modify the code here to generate other such contrastive examples, for other dataset combinations.

### B.1) TONE BANK

<blockquote style="border:2px solid #000; padding:12px 16px; border-radius:8px; background:#fff; margin:16px 0;">
  <div style="display:flex; gap:.6rem; align-items:flex-start;">
    <div style="font-size:1.1rem; line-height:1;">💬</div>
    <div>
      <div style="font-weight:700;">Original Prompt</div>
      <div>How can humor help diffuse tension during a disagreement?</div>
    </div>
  </div>
</blockquote>

| 🎨  | Tone           | What it sounds like                        | Example from the dataset                                                                                                                                                                          |
| --- | -------------- | ------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 🎓  | **Expert**     | Authoritative, theory-backed, structured.  | “Humor, when appropriately applied, can serve as a potent tool for diffusing tension during disagreements, a concept supported by various psychological theories and empirical studies.”          |
| 🛡️ | **Cautious**   | Hedged, risk-aware, emphasizes limits.     | “While our understanding of humor and its effects on interpersonal relationships is still evolving, it appears that humor may potentially play a role in diffusing tension during disagreements.” |
| 🤝  | **Empathetic** | Warm, validating, people-first.            | “I understand that disagreements can often be emotionally challenging and stressful… humor has a unique way of breaking down walls and creating a shared experience of laughter…”                 |
| 🗣️ | **Casual**     | Conversational, friendly, light on jargon. | “Hey there! … it feels like you're in a pressure cooker? Well, humor is like that magical safety valve that lets out some steam.”                                                                 |
| ⚡   | **Concise**    | Direct, minimal fluff; TL;DR vibe.         | “Humor can help diffuse tension during a disagreement by shifting the focus away from the conflict and reducing stress levels… foster a sense of camaraderie and mutual understanding.”           |


### B.2) DEBATE MIX

<blockquote style="border:2px solid #000; padding:12px 16px; border-radius:8px; background:#fff; margin:16px 0;">
  <div style="display:flex; gap:.6rem; align-items:flex-start;">
    <div style="font-size:1.1rem; line-height:1;">💬</div>
    <div>
      <div style="font-weight:700;">Original Prompt</div>
      <div>How do we reconcile the right to religious freedom with the need for societal cohesion and harmony?</div>
    </div>
  </div>
</blockquote>

| 🎭 | Debate style | What it is | Example from the dataset |
|---|---|---|---|
| 🧨 | **Reductio ad Absurdum** | Push the claim to an extreme to reveal contradictions. | “If we allow absolute religious freedom without any restrictions… any practice could be justified under religion.” :contentReference[oaicite:0]{index=0} |
| ⚖️ | **Appeal to Precedent** | Justify via constitutions, laws, or cases. | “In the U.S., the First Amendment protects belief, but *Reynolds v. United States (1878)* held practices can be limited.” :contentReference[oaicite:1]{index=1} |
| 🎯 | **Straw Man Reframing** | Restate an exaggerated version, then refute it. | “So you’re saying religious freedom is inherently a threat to harmony… that’s a gross oversimplification.” :contentReference[oaicite:2]{index=2} |
| 🧷 | **Burden of Proof Shift** | Demand the opponent disprove your position. | “There’s no evidence disproving that religious freedom supports harmony—can you definitively prove otherwise?” :contentReference[oaicite:3]{index=3} |
| 🔗 | **Analogy Construction** | Use a parallel to clarify the logic. | “Think of a symphony: many parts play freely, but harmonize for the whole—like freedom and cohesion.” :contentReference[oaicite:4]{index=4} |
| 🔀 | **Concession and Pivot** | Grant a minor point, then redirect to a stronger claim. | “Conflicts can occur, yes—but freedom and cohesion are not incompatible; the key is mutual respect.” :contentReference[oaicite:5]{index=5} |
| 📊 | **Empirical Grounding** | Cite data or studies as primary support. | “International law recognizes religious freedom; research (e.g., Grim & Finke) links it with civil liberties.” :contentReference[oaicite:6]{index=6} |
| 🧭 | **Moral Framing** | Anchor in ethics and shared values. | “This is a matter of justice, liberty, equality, and compassion—what ought we protect?” :contentReference[oaicite:7]{index=7} |
| 🪞 | **Refutation by Distinction** | Draw key differences that break an analogy or claim. | “Distinguish religious freedom from its misuse; distinguish cohesion from uniformity.” :contentReference[oaicite:8]{index=8} |
| 🔄 | **Circular Anticipation** | Preempt and answer likely objections. | “Some argue freedom breeds discord… others fear insular communities—but respect and rights limit such harms.” :contentReference[oaicite:9]{index=9} |


### B.3) Extract empathetic and cautious vectors from contrastive examples.

In [ ]:
tones_dataset_name = "amirali1985/tone_agnostic_questions"
tones_dataset = load_dataset(tones_dataset_name)

In [ ]:
def tokenize_text_response(tokenizer, prompt, response):
    tokenizer.padding_side='left'
    messages = [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response}
    ]
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        continue_final_message=None,
        return_tensors="pt",
        return_full_text=False,
        return_dict=True
    ).to('cuda')
    return tokenizer.decode(inputs['input_ids'][0])

In [ ]:
prompts = list(tones_dataset['train']['text'])

neutral_prompts = [tokenize_text(tokenizer, prompt, decode=True) for prompt in prompts]
cautious_prompts = [tokenize_text(tokenizer, prompt + " " + TONES['cautious'], decode=True) for prompt in prompts]
empathetic_prompts = [tokenize_text(tokenizer, prompt + " " + TONES['empathetic'], decode=True) for prompt in prompts]

In [ ]:
cautious_activations = activations_collector.collect_activations(
    pos_texts=cautious_prompts, neg_texts=neutral_prompts, config=steering_config
)

empathetic_activations = activations_collector.collect_activations(
    pos_texts=empathetic_prompts, neg_texts=neutral_prompts, config=steering_config
)

In [ ]:
# These are required steering vectors.
cautious_delta = cautious_activations['delta']
empathetic_delta = empathetic_activations['delta']
prompt_id = 95

In [ ]:
prompts[prompt_id]

In [ ]:
expert_prompts[prompt_id]

In [ ]:
generate_text(model, tokenizer, neutral_prompts[prompt_id], temperature=1.0, max_new_tokens=256)

In [ ]:
generate_text(model, tokenizer, cautious_prompts[prompt_id], temperature=1.0, max_new_tokens=256)

In [ ]:
tokens = tokenize_text(tokenizer=tokenizer, text=neutral_prompts[prompt_id])
generate_with_delta(
    llm,tokens,delta=cautious_delta, alpha=0.12*steering_config.alpha_tones, num_layers=9,layer_idx=15
)

In [ ]:
generate_with_delta(
    llm,tokens,delta=empathetic_delta, alpha=0.2*steering_config.alpha_tones, num_layers=9,layer_idx=15
)

### B.3 Multi-attribute steering

We add now a simple implementation of adding two attributes at once. Namely, we add in `expert` and 
`empathetic` tones. Note this is simple vector addition, and we can mix different weights.


In [ ]:
def generate_with_two_deltas(llm, tokens, delta_1, delta_2, alpha_1, alpha_2, layer_idx, num_layers=1):
    """
    Applies delta_1 and delta_2 on alternating layers within a window centered on layer_idx.
    Alternation is based on distance from the center (layer_idx): distance % 2 == 0 -> delta_1, else delta_2.
    For even num_layers, the lesser side (lower index) gets one extra layer, e.g. 4 -> [idx-2, idx-1, idx, idx+1].
    """
    n_layers = len(llm.model.layers)
    half = num_layers // 2

    if num_layers % 2 == 0:
        start = layer_idx - half
        end   = layer_idx + half - 1
    else:
        start = layer_idx - half
        end   = layer_idx + half

    # Clamp to valid layer range
    start = max(0, start)
    end   = min(n_layers - 1, end)
    num_edits = 200

    with llm.generate(tokens['input_ids'], max_new_tokens=256, pad_token_id=tokenizer.eos_token_id) as tracer:
        for curr_layer in range(start, end + 1):
            # Alternate by distance from center
            use_delta_2 = (abs(curr_layer - layer_idx) % 2 == 1)
            delta = (delta_2 * alpha_2) if use_delta_2 else (delta_1 * alpha_1)

            new_layer_output = llm.model.layers[curr_layer].output.clone() + delta
            llm.model.layers[curr_layer].output = new_layer_output


        with tracer.iter[0:200]:
            steps = list().save()
            if len(steps) == 0:
                steps.append(0)

            elif len(steps) < num_edits:
                steps.append(1)
                for curr_layer in range(start, end + 1):
                    curr_output = layers[curr_layer].output.clone()[:,-1,:]
                    use_delta_2 = (abs(curr_layer - layer_idx) % 1 == 1)
                    delta = delta_2 * alpha_2 + (delta_1 * alpha_1)
                    
                    layers[curr_layer].output[:,-1,:] = curr_output + delta * alpha
            else:
                steps.append(0)

        final_llm_output = llm.generator.output.save()

    return tokenizer.batch_decode(final_llm_output)[0].strip()

generate_with_two_deltas(
    llm,tokens,delta_1=cautious_delta, delta_2=empathetic_delta, alpha_1=0.51*steering_config.alpha_tones, 
    alpha_2=0.33*steering_config.alpha_tones, layer_idx=15, num_layers=9
)

## C.) So what's next?
* Check out our [paper](https://arxiv.org/abs/2505.24535) for more details, Note that for simplicity, we focused on simple CAA based steering here to illustrate activation engineering method. 

* See the official [$k$-steering github repo](https://github.com/withmartian/nonlinear_steering) for $k$ steering, and see if you can get it working! Let me know if you run into difficulties, or submit a PR to make it more robust.

* Can you come up with more examples of datasets and behaviors? For instance, [this paper]() introduces impatient, skeptical, confused and incoherent traits. And [this one]() introduces sycophancy, evil and hallucination. Finally, this paper adds the 5 basic emotions, namely joy, anger, fear, disgust and sadness.

* Can you think of applications where you can inject behaviors into models like this?